In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pydataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import warnings
warnings.filterwarnings("ignore")

import env
import wrangle
import split_scale

> 1. Load the tips dataset.

In [4]:
tips = pydataset.data('tips')

>> a. Create a column named tip_percentage. This should be the tip amount divided by the total bill.

In [8]:
tips['tip_percentage'] = (tips.tip)/(tips.total_bill)
tips

,total_bill,tip,sex,smoker,day,time,size,tip_percentage
1,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
2,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
3,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
4,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
5,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...,...
240,29.03,5.92,Male,No,Sat,Dinner,3,0.203927
241,27.18,2.00,Female,Yes,Sat,Dinner,2,0.073584
242,22.67,2.00,Male,Yes,Sat,Dinner,2,0.088222
243,17.82,1.75,Male,No,Sat,Dinner,2,0.098204


>> b. Create a column named price_per_person. This should be the total bill divided by the party size.

In [10]:
tips['price_per_person'] = (tips.total_bill)/(tips.size)
tips

,total_bill,tip,sex,smoker,day,time,size,tip_percentage,price_per_person
1,16.99,1.01,Female,No,Sun,Dinner,2,0.059447,0.007737
2,10.34,1.66,Male,No,Sun,Dinner,3,0.160542,0.004709
3,21.01,3.50,Male,No,Sun,Dinner,3,0.166587,0.009567
4,23.68,3.31,Male,No,Sun,Dinner,2,0.139780,0.010783
5,24.59,3.61,Female,No,Sun,Dinner,4,0.146808,0.011198
...,...,...,...,...,...,...,...,...,...
240,29.03,5.92,Male,No,Sat,Dinner,3,0.203927,0.013219
241,27.18,2.00,Female,Yes,Sat,Dinner,2,0.073584,0.012377
242,22.67,2.00,Male,Yes,Sat,Dinner,2,0.088222,0.010323
243,17.82,1.75,Male,No,Sat,Dinner,2,0.098204,0.008115


>> c. Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount? The tip percentage?

-The most important features for predicting the tip amount would be the size of the table, the time of day they eat, the day of the week, and the total bill

>> d. Use all the other numeric features to predict tip amount. Use select k best and recursive feature elimination to select the top 2 features. What are they?

>> e. Use all the other numeric features to predict tip percentage. Use select k best and recursive feature elimination to select the top 2 features. What are they?

>> f. Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?

> 2. Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

> 3. Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

> 4. Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).